In [52]:
import os
os.chdir("/content/drive/MyDrive/Colab/Deep_Dream_Keras_Implementation")

In [60]:
import numpy as np
import scipy
from IPython.display import Image

from keras.preprocessing.image import load_img , img_to_array , save_img
from keras.applications.inception_v3 import InceptionV3 , preprocess_input
from keras.utils import plot_model
import keras.backend as K

In [61]:
layer_contributions = {
    'conv2d_52': 25.,
    }

In [62]:
def preprocess_image(image_path):
  #util function to open , resize and format pictures
  #into appropriate tensors

  img = load_img(image_path)
  img = img_to_array(img)
  img = np.expand_dims(img , axis= 0)
  img = preprocess_input(img)
  return img

In [63]:
def deprocess_image(x):
  #util function to convert a tensor info a valid image
  x = np.copy(x)
  x = x.reshape(( x.shape[1] ,x.shape[2] , 3 ))
  x/=2
  x += 0.5
  x *= 255
  x = np.clip(x , 0 , 255).astype('uint8')

  return x

In [64]:
def resize_img(img , size):
  img = np.copy(img)
  factors = (1 ,
             float(size[0]) / img.shape[1] ,
             float(size[1]) / img.shape[2] ,
             1)

  return scipy.ndimage.zoom(img , factors , order= 1)

In [65]:
model = InceptionV3(weights='imagenet', include_top=False)
layer_dict = {layer.name : layer for layer in model.layers}

In [66]:
loss = K.variable(0.)
for layer_name, coeff in layer_contributions.items():
    output = layer_dict[layer_name].output
    scale = K.cast(K.prod(K.shape(output)), 'float32')
    loss += coeff * K.sum(K.square(output[:, 2:-2, 2:-2, :])) / scale

KeyError: ignored

In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

dream = model.input
grads = K.gradients(loss, dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)) , K.epsilon())

fetch_loss_grads = K.function([dream] , [loss , grads])

def eval_loss_grads(x):
  loss_value , grad_value =fetch_loss_grads([x])
  return loss_value , grad_value

def gradient_ascent(img , iterations , lr , max_loss = None):
  for i in range(iterations):
    loss_value , grad_value = eval_loss_grads(img)
    print(f'iter : {i} , loss_value : {loss_value}')
    if max_loss is not None and loss_value > max_loss:
      break
    img += lr * grad_value

  return img

TypeError: ignored

In [ ]:
#playing with these hyperparameters will also allow you to achieve new effects
lr = 0.01 #Gradient ascent step size
num_octave = 3 #number of scales at which to run gradient ascent
octave_scale = 1.4 #size ratio between scales
iterations = 20 #Number of ascent steps per scale
max_loss = 100.
image_path = 'sky.jpg'
result_path = 'sky_dream.jpg'

In [ ]:
img = preprocess_image(image_path)
original_shape = img.shape[1 : -1]
original_shape

(718, 1280)

In [ ]:
successive_shapes = [original_shape]
for i in range(1 , num_octave):
  shape = tuple([int(dim / (octave_scale**i)) for dim in original_shape])
  successive_shapes.append(shape)

successive_shapes.reverse()
original_img = np.copy(img)
shrunk_original_img = resize_img(img , successive_shapes[0])
print(shrunk_original_img.shape , original_img.shape)

(1, 366, 653, 3) (1, 718, 1280, 3)


In [ ]:
for shape in successive_shapes:
  print(f'proccessing shape {shape}')
  img = resize_img(img , shape)
  img = gradient_ascent(img , iterations , lr , max_loss)

  downscale_original_image = resize_img(original_img , shape)
  upscale_shrunk_original_image = resize_img(shrunk_original_img , shape)

  loss_details = downscale_original_image - upscale_shrunk_original_image
  img += loss_details

  shrunk_original_img = resize_img(original_img , shape)

save_img(result_path , deprocess_image(img))

proccessing shape (366, 653)


NameError: ignored